In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import itertools
import operator
from collections import Counter
from nltk import pos_tag
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from scipy import sparse as sp_sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict


from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
tags = pd.read_csv('./Tags.csv')

In [3]:
x = tags['Tag'].value_counts()

In [4]:
x = pd.DataFrame(x[x>35000])

In [5]:
x.head(10)

,Tag
javascript,124155
java,115212
c#,101186
php,98808
android,90659
jquery,78542
python,64601
html,58976
c++,47591
ios,47009


In [30]:
x = x.reset_index()

In [31]:
valid_ids = tags[tags['Tag'].isin(x['index'])]

In [32]:
len(valid_ids.Id)

947293

In [33]:
len(valid_ids.Id.unique())

759996

In [34]:
#Join all tags grouped by the question ID

valid_ids = valid_ids.groupby('Id').agg(lambda x: " ".join(x)).reset_index()


In [35]:
valid_ids.head(10)

,Id,Tag
0,120,sql
1,260,c#
2,330,c++
3,650,c#
4,930,c#
5,1010,c#
6,1040,c#
7,1760,c#
8,1970,sql
9,2120,sql


In [36]:
questions = pd.read_csv('./Questions.csv', encoding = 'latin-1')

In [37]:
ques_tags = pd.merge(left = questions, right = valid_ids, how = 'right', on = 'Id')

In [38]:
len(ques_tags)

759996

In [39]:
ques_tags = ques_tags[ques_tags.Score > 0]

In [40]:
len(ques_tags)

349136

In [41]:
test_tags = ques_tags[ques_tags.Score <= 2]

In [43]:
len(test_tags)


239433

In [44]:
len(ques_tags)

349136

In [45]:
ques_tags.to_pickle('top10_doc2vec')

In [46]:
ques_tags.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Tag
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,sql
1,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,c#
2,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,c++
3,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,<p>I would like the version property of my app...,c#
4,930,245.0,2008-08-04T00:47:25Z,NaN,28,How do I connect to a database and loop over a...,<p>What's the simplest way to connect and quer...,c#


In [47]:
# Add necessary stopwords

stopwords1 =  set(stopwords.words('english'))
stopwords1.add("I'm")
stopwords1.add("I've")
stopwords1.add('I')
stopwords1

{'I',
 "I'm",
 "I've",
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out

In [48]:
def cleanhtml(raw_html):
    '''
    Function to clean the data of hyperlinks and other text
    '''
    cleanr = re.compile('<.*?> ')
    cleanr1 = re.compile('<.*?>')
    cleantext1 = re.sub(cleanr, '', raw_html)
    cleantext = re.sub(cleanr1, '', cleantext1)
    return(cleantext)

In [49]:
def integer_frequency(list_str):
    '''
    Function to clean out line breaks and other not required text
    '''
    list_str = re.sub(r'^.*?https?:\/\/.*[\r\n]*', '', list_str, flags=re.MULTILINE)
    list_str = re.sub(r'^https?:\/\/.*[\r\n]*', '', list_str, flags=re.MULTILINE)
    list_str = list_str.replace('.', ' ')
    list_str = list_str.replace(',', ' ')
    list_str = list_str.replace('\n', ' ')
    list_str = list_str.replace('\r', ' ')
    list_str = list_str.replace('?', '')
    ls = [x for x in list_str.split(' ') if x not in stopwords1]
    return " ".join(ls)

In [50]:
def remove_stopwords(strn):
    return( " ".join([x for x in strn.split(" ") if x not in stopwords1]))

In [51]:
#Clean the whole dataset and combine Question Title & Body

ques_tags['Body'] = ques_tags['Body'].str.replace('<p>', '')
ques_tags['Body'] = ques_tags['Body'].str.replace('</p>', '')
ques_tags['Body'] = ques_tags['Body'].apply(lambda s: cleanhtml(s))
ques_tags['Body'] = ques_tags['Body'].apply(lambda s: integer_frequency(s))
ques_tags['ques'] = ques_tags['Title'] + ' ' + ques_tags['Body']
ques_tags= ques_tags[['Title','Id', 'ques','Tag']]


In [24]:
#Clean the whole dataset and combine Question Title & Body

test_tags['Body'] = test_tags['Body'].str.replace('<p>', '')
test_tags['Body'] = test_tags['Body'].str.replace('</p>', '')
test_tags['Body'] = test_tags['Body'].apply(lambda s: cleanhtml(s))
test_tags['Body'] = test_tags['Body'].apply(lambda s: integer_frequency(s))
test_tags['ques'] = test_tags['Title']
test_tags= test_tags[['Id', 'ques','Tag']]

test_tags['combined'] =test_tags['Tag'].astype(str) + " " + test_tags['ques'].astype(str)
test_tags['combined'] = test_tags['combined'].apply(lambda s: remove_stopwords(s))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [25]:
test_tags.head(100)

,Id,ques,Tag,combined
60,17430,Language Conversion Testing,c#,c# Language Conversion Testing
61,17670,Suitable alternative to CryptEncrypt,c++,c++ Suitable alternative CryptEncrypt
63,18920,"Can you set, or where is, the local document r...",html css,"html css Can set, is, local document root?"
64,19030,How to check set of files conform to a naming ...,python,python How check set files conform naming scheme
66,19970,Best way to keep an ordered list of windows (f...,javascript,javascript Best way keep ordered list windows ...
76,23370,Notification of drop in drag-drop in Windows,c# c++,c# c++ Notification drop drag-drop Windows
77,23620,What Javascript rich text editor will not brea...,javascript,javascript What Javascript rich text editor br...
82,25240,Inserting at the very end in FCKeditor,javascript,javascript Inserting end FCKeditor
101,28280,Can I maintain state between calls to a SQL Se...,sql,sql Can maintain state calls SQL Server UDF?
103,29040,Linq To SQL: Can I eager load only one field i...,sql,sql Linq To SQL: Can eager load one field join...


In [52]:
ques_tags.head()

,Title,Id,ques,Tag
0,ASP.NET Site Maps,120,ASP.NET Site Maps Has anyone got experience cr...,sql
1,Adding scripting functionality to .NET applica...,260,Adding scripting functionality to .NET applica...,c#
2,Should I use nested classes in this case?,330,Should I use nested classes in this case? work...,c++
3,Automatically update version number,650,Automatically update version number would like...,c#
4,How do I connect to a database and loop over a...,930,How do I connect to a database and loop over a...,c#


In [27]:
test_tags.Tag.head(100)

27                 php
60                  c#
61                 c++
63            html css
64              python
66          javascript
76              c# c++
77          javascript
82          javascript
101                sql
103                sql
115                php
121                 c#
128                php
132                 c#
143                sql
144              mysql
151                sql
157                php
160                sql
164             jquery
167                 c#
171         javascript
183                 c#
184                sql
196                c++
199                 c#
204                 c#
208                php
210                c++
            ...       
374               java
387                sql
389             python
393          sql mysql
394                 c#
402                php
405               java
408                php
409                 c#
411                css
415               html
416              mysql
418        

In [53]:
ques_tags['combined'] =ques_tags['Tag'].astype(str) + " " + ques_tags['ques'].astype(str)

In [54]:
ques_tags['combined'] = ques_tags['combined'].apply(lambda s: remove_stopwords(s))

In [55]:
len(ques_tags)

349136

In [50]:
ques_tags.head(10)

,Id,ques,Tag,combined
0,120,ASP.NET Site Maps Has anyone got experience cr...,sql,sql ASP.NET Site Maps Has anyone got experienc...
1,260,Adding scripting functionality to .NET applica...,c#,c# Adding scripting functionality .NET applica...
2,330,Should I use nested classes in this case? work...,c++,c++ Should use nested classes case? working co...
3,650,Automatically update version number would like...,c#,c# Automatically update version number would l...
4,930,How do I connect to a database and loop over a...,c#,c# How connect database loop recordset C#? Wha...
5,1010,"How to get the value of built, encoded ViewSta...",c#,"c# How get value built, encoded ViewState? nee..."
6,1040,How do I delete a file which is locked by anot...,c#,c# How delete file locked another process C#? ...
7,1760,.NET Unit Testing packages? Getting back bit ...,c#,c# .NET Unit Testing packages? Getting back bi...
8,1970,What language do you use for PostgreSQL trigge...,sql,sql What language use PostgreSQL triggers stor...
9,2120,Convert HashBytes to VarChar want get MD5 Hash...,sql,sql Convert HashBytes VarChar want get MD5 Has...


In [56]:
train,test = train_test_split(ques_tags)

In [57]:
np.shape(test)

(87284, 5)

In [58]:
train[['Id', 'Tag', 'ques','combined','Title']].to_csv('train_350000_10_doc2vec.csv', sep='\t', encoding='utf-8')
test[['Id', 'Tag', 'ques','combined','Title']].to_csv('test_350000_10_doc2vec.csv', sep='\t', encoding='utf-8')

In [53]:
ques_tags[['Id', 'Tag', 'ques','combined']].to_csv('350000_10_doc2vec.csv', sep='\t', encoding='utf-8')

In [97]:
for elem in ques_tags.iterrows():
    tup = elem[1]
    print(tup['Tag'].split(" "))

['sql']
['c#']
['c++']
['c#']
['c#']
['c#']
['c#']
['c#']
['sql']
['sql']
['c#']
['html']
['c#']
['sql']
['php', 'mysql']
['c++']
['c++']
['java']
['sql']
['php']
['c#']
['java']
['c#']
['html']
['c#']
['sql', 'mysql']
['c#']
['javascript']
['java']
['c++']
['css']
['c#']
['c#']
['sql']
['c#']
['c#']
['c++']
['java']
['php']
['c++', 'python']
['java']
['java', 'php']
['mysql']
['php']
['c#']
['c#']
['sql']
['php']
['c++']
['javascript']
['java']
['c#']
['c#']
['c#']
['java']
['c#']
['c#']
['c#']
['python']
['mysql']
['javascript']
['javascript']
['sql']
['java']
['c#', 'sql']
['html', 'css']
['sql']
['java']
['c++']
['c#']
['java']
['php']
['c#', 'sql']
['c#']
['css']
['sql']
['java']
['c++']
['c#']
['c#']
['jquery']
['c#']
['html']
['javascript']
['c++']
['java']
['c#']
['c#']
['c++']
['c++']
['c++']
['c++']
['android']
['c#']
['javascript']
['java']
['c++']
['c#']
['c#']
['java']
['css']
['c#']
['javascript']
['mysql']
['python']
['java']
['c#']
['sql']
['java']
['c#', 'sql']
['sql']

['c++']
['java']
['python']
['c#']
['java']
['c++']
['css']
['html', 'css']
['c#']
['c#']
['sql']
['c#']
['html']
['c++']
['java']
['c++', 'python']
['c#']
['c#']
['c++']
['php']
['javascript', 'html']
['html', 'css']
['c#']
['sql']
['javascript']
['c++']
['c#', 'python']
['sql']
['javascript']
['php']
['java']
['c#']
['html']
['java']
['java']
['php']
['sql']
['c#']
['python']
['html', 'css']
['java']
['java']
['java']
['c++']
['java']
['javascript']
['c++']
['c#', 'javascript']
['javascript', 'jquery', 'html']
['java']
['python']
['c#']
['python']
['c#']
['javascript']
['javascript', 'jquery']
['python']
['c#']
['mysql']
['java']
['sql']
['java']
['c#']
['c#']
['c#']
['java']
['java']
['c#']
['c#']
['mysql', 'sql']
['php']
['sql']
['html', 'css']
['c++']
['c++']
['java']
['c#']
['java']
['c#']
['c++']
['c++']
['php']
['php']
['java']
['java']
['php']
['python']
['php']
['php']
['php', 'javascript']
['c++']
['javascript', 'jquery']
['php']
['c#']
['c#']
['python']
['java']
['mysql']
[

['c#', 'c++']
['c#']
['c#']
['python']
['java']
['javascript']
['c++']
['php']
['c#']
['c#']
['jquery']
['c#']
['javascript']
['java']
['jquery']
['java']
['c#']
['c#']
['c++']
['c++']
['java']
['c#']
['c#']
['c#']
['c#']
['javascript', 'python']
['javascript', 'css']
['c++']
['python']
['c#']
['c#']
['c#']
['c++']
['c++']
['sql']
['mysql']
['c++']
['java']
['c#']
['c#']
['c#']
['c++']
['java']
['c#']
['jquery']
['javascript']
['sql']
['css']
['java']
['java']
['c++']
['c#']
['php']
['java']
['c#', 'java']
['python']
['java']
['python']
['python', 'sql']
['c++']
['javascript']
['java', 'c++']
['html', 'css']
['c++']
['c++']
['java']
['python']
['php']
['python']
['javascript']
['c#']
['c++']
['java']
['python']
['javascript', 'jquery', 'html']
['c#', 'sql']
['jquery']
['c++']
['c#']
['c#']
['c#']
['c#']
['javascript', 'jquery']
['ios']
['c#']
['java']
['c#']
['c#']
['javascript', 'jquery']
['c++']
['php']
['c#']
['ios']
['c#']
['c++']
['java']
['c++']
['c++']
['jquery']
['html']
['java

['c#', 'c++']
['html']
['c#']
['c#']
['javascript', 'jquery']
['javascript']
['php']
['sql']
['php']
['python']
['html', 'css']
['sql']
['php']
['java']
['c#']
['php']
['php']
['c#']
['c#']
['c++']
['c#']
['java']
['c#']
['c#']
['c#']
['sql']
['c#', 'c++']
['javascript', 'jquery']
['c++']
['java']
['javascript', 'jquery']
['java']
['python']
['c#']
['c#']
['jquery']
['javascript']
['c#']
['c#']
['mysql']
['python', 'mysql']
['python']
['javascript']
['c#']
['php']
['java']
['c++']
['c++']
['php']
['c#']
['python']
['java']
['javascript', 'html']
['javascript', 'jquery', 'css']
['java']
['javascript', 'jquery']
['sql']
['html', 'css']
['c#']
['c++']
['c++']
['html']
['python']
['javascript', 'jquery']
['c++']
['python']
['c++']
['javascript', 'jquery']
['c#']
['java']
['javascript']
['java']
['javascript']
['c++']
['ios']
['c#']
['c#']
['c#']
['python']
['java']
['c#']
['java']
['php', 'javascript', 'mysql']
['css']
['java']
['php']
['c++']
['python']
['python']
['c#']
['c++']
['c++']
[

['html', 'css']
['python']
['javascript']
['python']
['php']
['java']
['c#', 'java']
['javascript']
['python']
['php']
['c#']
['c#']
['mysql']
['c++']
['python']
['python']
['php', 'mysql']
['c#']
['c++']
['java']
['c#']
['sql']
['javascript']
['c#']
['android']
['python']
['sql', 'mysql']
['python']
['html', 'css']
['python']
['python']
['java']
['mysql']
['c#']
['html']
['python']
['jquery']
['python', 'mysql']
['python']
['php']
['c++']
['java']
['jquery']
['c#']
['c#', 'mysql']
['c#']
['javascript', 'html']
['python']
['python']
['php']
['c++']
['javascript', 'jquery']
['css']
['php']
['java']
['c++']
['c#']
['java']
['c++']
['jquery']
['c#']
['c#']
['sql']
['c#']
['css']
['php']
['java']
['java']
['python']
['c#']
['python']
['java']
['java']
['c#']
['javascript']
['php', 'mysql']
['javascript', 'html']
['sql']
['c++', 'python']
['java']
['c#']
['c#']
['javascript']
['python']
['jquery']
['mysql']
['c#']
['javascript', 'html']
['c#']
['jquery']
['c#']
['c#']
['css']
['python']
['p

...

['ios']
['javascript']
['javascript', 'php', 'jquery', 'mysql']
['c#']
['java']
['ios']
['javascript', 'jquery', 'html', 'css']
['android']
['css']
['javascript']
['java']
['c#']
['css']
['c#']
['javascript']
['php']
['php', 'mysql']
['c#']
['javascript', 'python']
['ios']
['javascript']
['c#']
['android']
['css']
['c++']
['c++']
['c++']
['java']
['html', 'css']
['python']
['c++']
['c#']
['javascript', 'jquery', 'html', 'css']
['c#']
['javascript', 'jquery', 'html', 'css']
['php']
['android']
['javascript']
['java']
['java']
['java']
['ios']
['python']
['java', 'android']
['python']
['java']
['c++']
['android']
['python']
['ios']
['java']
['php', 'mysql']
['php']
['ios']
['c++']
['python']
['ios']
['html']
['java']
['c++']
['java']
['python']
['c#']
['python']
['c#']
['ios']
['c++']
['jquery']
['javascript']
['python']
['ios']
['python']
['mysql', 'sql']
['c++']
['java']
['java']
['java']
['java']
['python']
['php']
['python']
['javascript', 'html', 'css']
['php', 'android', 'ios']
['p

['php', 'android', 'mysql']
['android']
['java']
['css']
['php', 'mysql']
['python']
['c++']
['c++']
['java']
['c#']
['php']
['javascript']
['php']
['python']
['javascript']
['javascript']
['android']
['c++']
['c++']
['java']
['javascript', 'html']
['android']
['c#']
['ios']
['python']
['javascript']
['java']
['php']
['javascript', 'jquery', 'html']
['c++']
['javascript', 'jquery', 'html']
['c#']
['c++']
['javascript']
['android']
['c#']
['c#']
['php']
['java']
['python']
['c#']
['c++']
['c++']
['java', 'android']
['c++']
['java']
['sql']
['javascript']
['c#']
['jquery', 'html']
['php']
['android']
['javascript', 'jquery', 'html', 'css']
['python']
['c#']
['python']
['php']
['javascript']
['java']
['php']
['ios']
['c++']
['android']
['java']
['c++']
['javascript', 'html']
['android']
['android']
['javascript']
['android']
['javascript']
['c#']
['javascript', 'jquery']
['javascript']
['c++']
['android']
['android']
['javascript']
['java']
['javascript', 'jquery']
['ios']
['javascript', 

['python']
['c#']
['python']
['java']
['c++']
['c#']
['java']
['javascript', 'jquery', 'ios']
['c#']
['ios']
['php']
['python']
['python']
['java']
['java']
['android']
['c#']
['python']
['html', 'css']
['php', 'mysql']
['c++']
['c++']
['android']
['java']
['php']
['c++']
['c#']
['android']
['c++']
['java']
['python']
['java']
['python']
['html', 'css']
['java']
['javascript']
['python']
['c++']
['c++']
['java']
['java']
['javascript']
['mysql']
['java']
['java']
['python']
['javascript']
['python']
['mysql', 'sql']
['java']
['javascript', 'html', 'css']
['javascript']
['android']
['css']
['python']
['jquery']
['python']
['python']
['javascript', 'android']
['mysql', 'sql']
['python']
['sql']
['sql']
['python']
['java', 'android']
['javascript']
['c++']
['javascript', 'jquery']
['c#']
['javascript', 'jquery']
['c#']
['html', 'css']
['c#']
['javascript']
['java']
['android']
['c++']
['javascript', 'jquery', 'html']
['python']
['c#']
['javascript']
['c#']
['javascript']
['android']
['jav

['c++']
['javascript', 'html']
['python']
['python']
['python']
['ios']
['android']
['sql']
['java', 'android']
['c#']
['jquery']
['c#']
['java']
['c++']
['javascript']
['c++']
['python']
['javascript']
['java']
['c++']
['php']
['python']
['android']
['python']
['ios']
['java']
['android']
['javascript']
['java']
['mysql', 'sql']
['java']
['c++']
['python']
['php', 'mysql', 'sql']
['ios']
['android']
['python']
['css']
['javascript']
['javascript']
['javascript', 'html']
['c#']
['javascript']
['python']
['html', 'css']
['android']
['c++']
['c++']
['c++']
['c#']
['ios']
['c++']
['javascript', 'jquery']
['android']
['java']
['c++']
['c++']
['javascript']
['php', 'html']
['android']
['ios', 'css']
['javascript']
['sql']
['c#']
['java', 'android']
['c#']
['c#']
['java', 'css']
['c#']
['javascript']
['python']
['javascript', 'java']
['javascript', 'jquery']
['ios']
['python']
['ios']
['c#']
['java']
['ios']
['java']
['python']
['c#']
['c++']
['c++']
['c#']
['android']
['sql']
['html', 'css'

['java', 'android']
['c++']
['python']
['c++']
['python']
['python']
['python']
['php']
['c#']
['ios']
['c++']
['c++']
['php']
['java']
['java']
['python']
['sql']
['c++']
['python']
['python']
['javascript']
['c++']
['python']
['c++']
['python']
['python']
['javascript', 'php']
['javascript']
['python']
['java']
['python']
['c++']
['python']
['python']
['java']
['javascript', 'css']
['html', 'css']
['c++']
['c++']
['css']
['javascript']
['javascript']
['javascript', 'html']
['android']
['c#']
['java']
['c++']
